In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format('parquet').load('abfss://bronze@etedbxcrop.dfs.core.windows.net/products')

In [0]:
df = df.drop('_rescued_data')
df.display()

In [0]:
df.createOrReplaceTempView('products')

In [0]:
%sql
CREATE OR REPLACE FUNCTION dbx_cata.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.85

In [0]:
%sql
SELECT product_id, dbx_cata.bronze.discount_func(price) d_price, price 
FROM products

In [0]:
df = df.withColumn('d_price', expr('dbx_cata.bronze.discount_func(price)'))
df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION dbx_cata.bronze.dis_funct(price DOUBLE)
RETURNS DOUBLE
LANGUAGE PYTHON
AS
$$
  return price * 0.85
$$

-- This is the python equivalent of the SQL codeblock written above

In [0]:
df.withColumn('dd_price', expr('dbx_cata.bronze.dis_funct(price)')).display()

In [0]:
display(df)

In [0]:
df.write.format('delta').mode('append').save('abfss://silver@etedbxcrop.dfs.core.windows.net/products')